In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sklearn
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.decomposition import PCA
import time
import sys
import math
import numpy as np
import random
import joblib
from tqdm import tqdm_notebook

import Config
import Dataloader as DL
import HD_basis as HDB
import HD_encoder as HDE
import HD_classifier as HDC


In [3]:
dl = DL.Dataloader()
nFeatures, nClasses, traindata, trainlabels, testdata, testlabels = dl.getParam()

Loading dataset MNIST from MNIST
Loading train data... train data of shape (60000, 784) loaded
Loading test data...  test  data of shape (10000, 784) loaded
Data Loaded. Num of features = 784 Num of Classes = 10

In [4]:
param = Config.config
param["nFeatures"] = nFeatures
param["nClasses"] = nClasses

In [5]:
################# VANILLA #################
hdb = HDB.HD_basis(HDB.Generator.Vanilla, param)
basis = hdb.getBasis()
bid = hdb.getParam()["id"]
# Update param with bid
param = hdb.getParam()
print(bid)

Generating vanilla HD basis of shape... 

(2500, 16)
Encoding time: 0.04783749580383301 
Dumping basis into file: base_8430.pkl 



8430


In [6]:
# Retrieve info upto basis generator, given correct bid
#bid = 6679
#basis, param = HDB.loadBasis("base_%d.pkl"%bid)

In [ ]:
hde = HDE.HD_encoder(basis)

trainencoded = hde.encodeData(traindata)
HDE.saveEncoded(trainencoded, trainlabels, bid, "train")

testencoded = hde.encodeData(testdata)
HDE.saveEncoded(testencoded, testlabels, bid, "test")

Encoding data of shape (362, 16)


In [ ]:
# Retrieve info upto encoder, given correct bid
#bid = 6679
#basis, param = HDB.loadBasis("base_%d.pkl"%bid)
#trainencoded, trainlabels = HDE.loadEncoded("encoded_%d_train.pkl"%bid)
#testencoded, testlabels = HDE.loadEncoded("encoded_%d_test.pkl"%bid)

In [10]:
# train data 
def train(hdc, traindata, trainlabels, testdata, testlabels, param = Config.config):
    train_acc = []
    test_acc = []
    for _ in tqdm_notebook(range(param["epochs"]), desc='epochs'):
        train_acc.append(hdc.fit(traindata, trainlabels, param))
        test_acc.append(hdc.test(testdata, testlabels))
        if len(train_acc) % 5 == 0:
            print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
        if train_acc[-1] == 1:
            print("Train: %f \t \t Test: %f"%(train_acc[-1], test_acc[-1]))
            break
    return np.asarray(train_acc), np.asarray(test_acc)

In [11]:
def dump_log(param, train_acc, test_acc, filename):
    joblib.dump((param, train_acc, test_acc), open(filename, "wb"), compress=True)

In [ ]:
# Should have 90%
hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = train(hdc, trainencoded, trainlabels, testencoded, testlabels)
dump_log(param, train_acc, test_acc, "Baseline_"+param["id"]+".pickle")

In [6]:
################# BAKLAVA #################
param["nLayers"] = 1
hdb = HDB.HD_basis(HDB.Generator.Baklava, param, noise = False)
basis = hdb.getBasis()
bid = hdb.getParam()["id"]
# Update param with bid
param = hdb.getParam()
print(bid)

TypeError: __init__() got an unexpected keyword argument 'noise'

In [7]:
# Retrieve info upto basis generator, given correct bid
bid = 2775
basis, param = HDB.loadBasis("base_%d.pkl"%bid)

In [8]:
hde = HDE.HD_encoder(basis, False)

trainencoded = hde.encodeData(traindata)
HDE.saveEncoded(trainencoded, trainlabels, bid, "train")

testencoded = hde.encodeData(testdata)
HDE.saveEncoded(testencoded, testlabels, bid, "test")

Encoding data of shape (60000, 784)


Time spent: 1089 sec
Dumping data into encoded_2775_train.pkl 
Encoding data of shape (10000, 784)


Time spent: 183 sec
Dumping data into encoded_2775_test.pkl 


'encoded_2775_test.pkl'

In [12]:
# Should have 96%
hdc = HDC.HD_classifier(param["D"], param["nClasses"], bid)
train_acc, test_acc = train(hdc, trainencoded, trainlabels, testencoded, testlabels)
dump_log(param, train_acc, test_acc, "ConvHD_"+param["id"]+".pickle")

c:\program files (x86)\microsoft visual studio\shared\python37_64\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


Fitting with configuration: [('one_shot', 0), ('dropout', 0), ('lr', 0.037)] 


Train: 0.831450 	 	 Test: 0.784400
Train: 0.844517 	 	 Test: 0.786800
Train: 0.850767 	 	 Test: 0.782000
Train: 0.854200 	 	 Test: 0.786100
Train: 0.856133 	 	 Test: 0.779900
Train: 0.856717 	 	 Test: 0.784000
Train: 0.858950 	 	 Test: 0.780800
Train: 0.859000 	 	 Test: 0.777500
Train: 0.859483 	 	 Test: 0.780600
Train: 0.860867 	 	 Test: 0.776600
Train: 0.861383 	 	 Test: 0.780300
Train: 0.862733 	 	 Test: 0.784200
Train: 0.862867 	 	 Test: 0.781200
Train: 0.862317 	 	 Test: 0.781000
Train: 0.863783 	 	 Test: 0.776300
Train: 0.864600 	 	 Test: 0.782300
Train: 0.863283 	 	 Test: 0.777800
Train: 0.862283 	 	 Test: 0.778100
Train: 0.863300 	 	 Test: 0.785100
Train: 0.864733 	 	 Test: 0.778000
Train: 0.863583 	 	 Test: 0.781700
Train: 0.863000 	 	 Test: 0.780200
Train: 0.863450 	 	 Test: 0.779400
Train: 0.864833 	 	 Test: 0.781100
Train: 0.864900 	 	 Test: 0.784400
Train: 0.864550 	 	 Test: 0.782200


KeyboardInterrupt: 

In [ ]:
## Random code
e = np.empty((2,2,2), dtype = int)
q = np.empty((2,2,2), dtype = int)
f = np.asarray([[[1,2],[3,4]],[[5,6],[7,8]]])
g = f * 10
np.concatenate((e, f, g), axis = 2)


tl = np.asarray([10,11,12,13,14,15,16,17,18])
tr = np.asarray([20,21,22,23,24,25,26,27,28])
bl = np.asarray([30,31,32,33,34,35,36,37,38])
br = np.asarray([40,41,42,43,44,45,46,47,48])
#print(tl.shape)
#print(tl[0:2,0:2])
e = HDB.bak_extend(tl,tr,bl,br, 3, 3)
f = []
f.append(e)
f.append(e)
f.append(e)
np.concatenate(f, axis = 2)